In [33]:
## Import Liberaries

import sqlite3 as sql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [34]:
## Read the file

df = pd.read_csv(r"C:\Users\tanvi\Desktop\resume all project code\SQL project\StudentsPerformance.csv") ## reads the dataframe
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [35]:
## print the columns of all dataframes

df.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [36]:
# Rename the columns
df = df.rename(columns={
    'gender': 'gender',
    'race/ethnicity': 'race_ethnicity',
    'parental level of education': 'parental_level_of_education',
    'test preparation course': 'test_preparation_course',
    'math score': 'math_score',
    'reading score': 'reading_score',
    'writing score': 'writing_score'
}, inplace=False)
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [37]:
# Create a SQLite engine
engine = create_engine('sqlite://', echo=False)

# Export the dataframe as a table 'students' to the SQLite engine
df.to_sql("students", con=engine, index=False)

1000

In [38]:
## Test the code by select the complete table
sql = '''
SELECT * FROM students
LIMIT 5
'''

df_sql = pd.read_sql_query(sql, con=engine)
df_sql

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [39]:
sql = '''
SELECT 
    row_number() OVER (ORDER BY total DESC) AS row_number,
    total
FROM (
    SELECT 
        (math_score + reading_score + writing_score) AS total
    FROM students
) a
LIMIT 5
'''

df_sql = pd.read_sql_query(sql, con=engine)
df_sql

,row_number,total
0,1,300
1,2,300
2,3,300
3,4,299
4,5,297


In [40]:
#Gender and Race/Ethnicity Breakdown
sql = '''
SELECT 
    gender,
    race_ethnicity,
    COUNT(*) AS count_of_races
FROM students
GROUP BY gender, race_ethnicity
ORDER BY gender, race_ethnicity
'''

df_sql = pd.read_sql_query(sql, con=engine)
df_sql


,gender,race_ethnicity,count_of_races
0,female,group A,36
1,female,group B,104
2,female,group C,180
3,female,group D,129
4,female,group E,69
5,male,group A,53
6,male,group B,86
7,male,group C,139
8,male,group D,133
9,male,group E,71


In [41]:
#Detailed Performance Metrics by Gender and Race/Ethnicity
sql = '''
SELECT 
    gender,
    race_ethnicity,
    AVG(math_score) AS avg_math_score,
    AVG(reading_score) AS avg_reading_score,
    AVG(writing_score) AS avg_writing_score,
    AVG(math_score + reading_score + writing_score) AS avg_total_score,
    COUNT(*) AS student_count
FROM students
GROUP BY gender, race_ethnicity
ORDER BY gender, race_ethnicity
'''

df_sql = pd.read_sql_query(sql, con=engine)
print(df_sql)


   gender race_ethnicity  avg_math_score  avg_reading_score   
0  female        group A       58.527778          69.000000  \
1  female        group B       61.403846          71.076923   
2  female        group C       62.033333          71.944444   
3  female        group D       65.248062          74.046512   
4  female        group E       70.811594          75.840580   
5    male        group A       63.735849          61.735849   
6    male        group B       65.930233          62.848837   
7    male        group C       67.611511          65.424460   
8    male        group D       69.413534          66.135338   
9    male        group E       76.746479          70.295775   

   avg_writing_score  avg_total_score  student_count  
0          67.861111       195.388889             36  
1          70.048077       202.528846            104  
2          71.777778       205.755556            180  
3          75.023256       214.317829            129  
4          75.536232       222.

In [42]:
# Calculate average total score
avg_total_score = df[['math_score', 'reading_score', 'writing_score']].sum(axis=1).mean()

sql = f'''
SELECT
    gender,
    SUM(CASE WHEN total > {avg_total_score} THEN 1 ELSE 0 END) AS above_avg,
    SUM(CASE WHEN total <= {avg_total_score} THEN 1 ELSE 0 END) AS below_avg,
    race_ethnicity,
    COUNT(*) AS total_students
FROM (
    SELECT 
        gender,
        race_ethnicity,
        (math_score + reading_score + writing_score) AS total
    FROM students
) a
GROUP BY gender, race_ethnicity
ORDER BY gender, race_ethnicity
'''

df_sql = pd.read_sql_query(sql, con=engine)
df_sql


,gender,above_avg,below_avg,race_ethnicity,total_students
0,female,13,23,group A,36
1,female,51,53,group B,104
2,female,104,76,group C,180
3,female,83,46,group D,129
4,female,49,20,group E,69
5,male,18,35,group A,53
6,male,30,56,group B,86
7,male,62,77,group C,139
8,male,69,64,group D,133
9,male,47,24,group E,71


In [43]:
## count of students more than the avg.

sql='''
select gender,
count (distinct race_ethnicity) as count_of_races from students
group by 1


''';

#round(avg(b.total),2) as avg_vale
    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql

,gender,count_of_races
0,female,5
1,male,5
